In [20]:
import pandas as pd
import numpy as np

In [21]:
df=pd.read_parquet('../features/loan_default_gen')

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [23]:
spark = (
        SparkSession.builder.master("local[*]")
        .appName(name="unittest")
        .config("spark.ui.enabled", "false")
        .config(
            "spark.driver.bindAddress",
            "0.0.0.0",
        ).getOrCreate())

## time series feature

In [24]:
df

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default,user_id,txn_datetime
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0,FOeIRQnsLbpjZvurkMwi,2021-11-10 00:17:55
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0,kyHoSmrOEWTzDRlpqBVQ,2022-08-27 19:45:08
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1,znUAgCQIhmGpWofycevH,2022-09-06 09:18:16
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0,PTwkdNvmUazZjcLElrun,2022-10-17 06:17:40
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0,rLTKXCBusQyfAMZpzHoS,2021-01-18 18:27:53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255342,8C6S86ESGC,19,37979,210682,541,109,4,14.11,12,0.85,Bachelor's,Full-time,Married,No,No,Other,No,0,muDptexSOYJqzPHiFcLZ,2022-04-11 11:17:09
255343,98R4KDHNND,32,51953,189899,511,14,2,11.55,24,0.21,High School,Part-time,Divorced,No,No,Home,No,1,mewsjcBIpPJFRauTdtNk,2021-09-07 17:19:19
255344,XQK1UUUNGP,56,84820,208294,597,70,3,5.29,60,0.50,High School,Self-employed,Married,Yes,Yes,Auto,Yes,0,SBxlRjhVgndCAtUQuskf,2023-04-27 03:18:58
255345,JAO28CPL4H,42,85109,60575,809,40,1,20.90,48,0.44,High School,Part-time,Single,Yes,Yes,Other,No,0,vyHrhRlCnwjfYkZuBMGm,2023-06-10 10:29:55


In [25]:
user_field='user_id'
time_field='txn_datetime'

In [26]:
ts_features=['Age','Income','CreditScore',"Default",'InterestRate','LoanTerm','LoanAmount']

In [27]:
df_ts_feature=df[[*ts_features,user_field,time_field]]

In [28]:
df_previous_feature=df_ts_feature.merge(
    df_ts_feature,
    on='user_id'
).loc[lambda x:x.txn_datetime_x>x.txn_datetime_y].groupby([user_field,f'{time_field}_x']).agg(
    previous_income=('Income_y',list),
    previous_age=('Age_y',list),
    previous_creditscore=('CreditScore_y',list),
    previous_default=('Default_y',list),
    previous_loan_amount=('LoanAmount_y',list),
    previous_loan_term=('LoanTerm_y',list),
)

In [29]:
def statistic_for_list(list_val:list, stats_function):
    return stats_function(list_val)

In [30]:
stats_functions={
    'mean': np.mean,
    'max': np.max,
    'min': np.min,
    'median':np.median,
    'quantile_25': lambda x: np.quantile(x,.25),
    'quantile_75': lambda x: np.quantile(x,.75),
    'count':len
}

In [31]:
df_previous_feature=df_previous_feature.reset_index(
    )

In [32]:
for col in ['previous_income','previous_age','previous_creditscore','previous_default','previous_loan_amount','previous_loan_term']:
    for key in stats_functions:
        stats_function=stats_functions[key]
        df_previous_feature[f'{key}_{col}']=df_previous_feature[col].apply(lambda x:statistic_for_list(x,stats_functions[key]))
        

In [35]:
df_previous_feature[['user_id','txn_datetime_x','mean_previous_income','quantile_75_previous_loan_amount','mean_previous_loan_term','count_previous_loan_amount']].rename({'txn_datetime_x':'txn_datetime'},axis=1)

,user_id,txn_datetime,mean_previous_income,quantile_75_previous_loan_amount,mean_previous_loan_term,count_previous_loan_amount
0,ABWCfzhgUTalnbJPRHui,2022-05-13 05:28:49,140051.000000,147774.00,24.0,1
1,ABWCfzhgUTalnbJPRHui,2023-01-11 19:04:33,138191.500000,114575.00,42.0,2
2,ABWCfzhgUTalnbJPRHui,2023-10-07 10:53:09,101404.666667,118521.00,36.0,3
3,ABWCfzhgUTalnbJPRHui,2023-12-02 01:43:05,98441.250000,131102.25,39.0,4
4,ABXYUkpfseHraCwnNuLK,2022-07-19 12:33:21,99547.000000,105603.00,60.0,1
...,...,...,...,...,...,...
205640,zyvcawDpXfrdjbKSoPUJ,2023-09-09 23:50:12,40042.750000,216466.00,27.0,4
205641,zyxNcmiKEoIudVlYnSea,2021-10-01 11:01:54,84672.000000,248752.00,48.0,1
205642,zyxNcmiKEoIudVlYnSea,2022-05-10 17:45:55,90217.000000,188322.75,30.0,2
205643,zyxNcmiKEoIudVlYnSea,2023-01-26 04:30:23,69188.333333,214611.00,40.0,3


In [15]:
df_previous_spark=spark.createDataFrame(df_previous_feature)

In [16]:
df_previous_spark=df_previous_spark.withColumnRenamed(
    'txn_datetime_x','txn_datetime'
)

In [17]:
df_previous_spark

DataFrame[user_id: string, txn_datetime: timestamp, previous_income: array<bigint>, previous_age: array<bigint>, previous_creditscore: array<bigint>, previous_default: array<bigint>, previous_loan_amount: array<bigint>, previous_loan_term: array<bigint>, mean_previous_income: double, max_previous_income: bigint, min_previous_income: bigint, median_previous_income: double, quantile_25_previous_income: double, quantile_75_previous_income: double, count_previous_income: bigint, mean_previous_age: double, max_previous_age: bigint, min_previous_age: bigint, median_previous_age: double, quantile_25_previous_age: double, quantile_75_previous_age: double, count_previous_age: bigint, mean_previous_creditscore: double, max_previous_creditscore: bigint, min_previous_creditscore: bigint, median_previous_creditscore: double, quantile_25_previous_creditscore: double, quantile_75_previous_creditscore: double, count_previous_creditscore: bigint, mean_previous_default: double, max_previous_default: big

In [ ]:
df_timeserie_feature=None

In [18]:
df_timeserie_feature.write.mode('overwrite').parquet('../features/ts_feature')

24/03/28 02:56:30 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/03/28 02:56:31 WARN TaskSetManager: Stage 0 contains a task of very large size (9086 KiB). The maximum recommended task size is 1000 KiB.
24/03/28 02:56:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [19]:
spark.stop()